In [56]:
import alpaca_trade_api as tradeapi
from datetime import date
import time
import datetime
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pytz import timezone
import pandas_ta as ta


BASE_URL = 'https://paper-api.alpaca.markets'
PUB_KEY = 'PKAX56PAOU4925CJJ9BP'
SEC_KEY = 'mHmA1VzpYPQ4sgFbBarM24rfUxUwXTJDfJXkSlWh'
api = tradeapi.REST(key_id= PUB_KEY, secret_key=SEC_KEY, base_url=BASE_URL)

In [49]:
ticker = "FAS"

In [50]:
## Getting the latest data point ##
last_price = api.get_last_trade(ticker).price
last_timestamp = api.get_last_trade(ticker).timestamp
print("timestamp: {}, price: {}".format(last_timestamp, last_price))

timestamp: 2022-01-03 12:08:55.435559315-05:00, price: 135.11


In [69]:
## this is all the data I need for the calculation ##
data = api.get_barset(ticker, 'day', limit=60).df

In [97]:
## Making a function that takes in ticker and outputs df with ti ##
def getDF(ticker, LIMIT):
    # Load data
    df = api.get_barset(ticker, 'day', limit=LIMIT).df
    df = df.set_axis(['open', 'high', 'low', 'close', 'volume'], axis=1, inplace=False)

    # add data points
    df['sma10'] = df.ta.sma(length=10)
    df['williams'] = df.ta.willr()

    df = df[[
            'open','close','sma10','williams'
            ]]

    df = df.dropna()
    
    return df

df = getDF(ticker, 60)
print(df.tail())
print(df.iloc[-1]['williams'])

                             open    close     sma10   williams
time                                                           
2021-12-28 00:00:00-05:00  132.48  132.620  127.3530 -13.657906
2021-12-29 00:00:00-05:00  133.60  132.140  127.8620 -15.725980
2021-12-30 00:00:00-05:00  133.07  131.270  128.1120 -19.474364
2021-12-31 00:00:00-05:00  130.72  130.700  127.9480 -21.930202
2022-01-03 00:00:00-05:00  133.56  134.562  128.9952  -8.522680
-8.522679983354141


In [77]:
## check market time ##
def time_to_market_close():
    clock = api.get_clock()
    return (clock.next_close - clock.timestamp).total_seconds()/60

minutes = time_to_market_close()
print("The market closes in {} minutes.".format(minutes))

The market closes in 156.99626868333334 minutes.


In [94]:
## check current positions ##
# Get a list of all of our positions.
portfolio = api.list_positions()
# Print the quantity of shares for each position.
for position in portfolio:
    print("I own {} shares of {}.".format(position.qty, position.symbol))

I own 20 shares of YINN.
I own 20 shares of NAIL.
I own 20 shares of LABU.


In [96]:
## list orders and details ##
orders = api.list_orders()
print("{} orders in.".format(len(orders)))
for o in orders:
    print("Sell {} shares of {} on {}.".format(o.qty, o.symbol, o.time_in_force))

5 orders in.
Sell 10 shares of TECL on cls.
Sell 10 shares of NAIL on cls.
Sell 10 shares of YINN on cls.
Sell 10 shares of LABU on cls.
Sell 10 shares of LABU on cls.


In [127]:
## check to see if any of tickers have buy signals ##
def should_buy(tickers):
    des = []
    for ticker in tickers:
        data = getDF(ticker, 60)
        if data.iloc[-1]['williams'] < -80:
            des.append([ticker,True])
        else:
            des.append([ticker,False])
    return des

tickers = ["FAS","TNA","SOXL","TECL","LABU","YINN","DPST","NAIL"]
des1 = should_buy(tickers)
for d in des1:
    print("Buying descision: {} for ticker: {}.".format(d[1], d[0]))


Buying descision: False for ticker: FAS.
Buying descision: False for ticker: TNA.
Buying descision: False for ticker: SOXL.
Buying descision: False for ticker: TECL.
Buying descision: False for ticker: LABU.
Buying descision: False for ticker: YINN.
Buying descision: False for ticker: DPST.
Buying descision: False for ticker: NAIL.


In [191]:
## check to see if any of tickers have sell signals ##
def should_sell(tickers):
    des = []
    for ticker in tickers:
        data = getDF(ticker, 60)
        if data.iloc[-1]['williams'] > -20:
            des.append([ticker,True])
        else:
            des.append([ticker,False])
    return des

tickers = ["FAS","TNA","SOXL","TECL","LABU","YINN","DPST","NAIL"]
des2 = should_buy(tickers)
for d in des2:
    print("Selling descision: {} for ticker: {}.".format(d[1], d[0]))

Selling descision: True for ticker: FAS.
Selling descision: True for ticker: TNA.
Selling descision: True for ticker: SOXL.
Selling descision: True for ticker: TECL.
Selling descision: False for ticker: LABU.
Selling descision: False for ticker: YINN.
Selling descision: True for ticker: DPST.
Selling descision: False for ticker: NAIL.


In [129]:
## check to see if there is a position in ticker ##
def does_pos(ticker):
    portfolio = api.list_positions()
    pos_owned = []
    for position in portfolio:
        pos_owned.append(position.symbol)
    state = ticker in pos_owned
    return state

dwo = does_pos('YINN')
print(dwo)

True


In [136]:
## if we do not own the ticker, buy order ##
account = api.get_account()
for i in des1:
    if i[1] == True and does_pos(i[0]) == False:
        
        # figuring out how many shares to buy #
        last_price = api.get_last_trade(i[0]).price
        order_amount = int(((float(account.buying_power)/len(tickers))*.95)/last_price)
        
        # Buy a stock
        api.submit_order(
          symbol=i[0]; # Replace with the ticker of the stock you want to buy
          qty=order_amount,
          side='buy',
          type='market', 
          time_in_force='day' # day
        )
        
        print("place buy order for {} and {} shares.".format(i[0], order_amount))

In [131]:
des1

[['FAS', False],
 ['TNA', False],
 ['SOXL', False],
 ['TECL', False],
 ['LABU', False],
 ['YINN', False],
 ['DPST', False],
 ['NAIL', False]]

In [134]:

## if we do own the ticker, sell order ##
for i in des2:
    if i[1] == True and does_pos(i[0]) == True:
        
        # how many shares to sell #
        order_amount = int(api.get_position(ticker).qty)
        
        api.submit_order(
          symbol=i[0]; # Replace with the ticker of the stock you want to buy
          qty=order_amount,
          side='sell',
          type='market', 
          time_in_force='gtc' # Good 'til cancelled
        )
        
        print("place sell order for {} and {} shares.".format(i[0], order_amount))

In [135]:
des2

[['FAS', True],
 ['TNA', True],
 ['SOXL', True],
 ['TECL', True],
 ['LABU', False],
 ['YINN', False],
 ['DPST', True],
 ['NAIL', False]]

In [198]:
## the start of the program will update us on a few things ##
account = api.get_account()
print("Account number: {}".format(account.account_number))
print("Buying power: ${}".format(account.buying_power))
print("Cash available: ${}".format(account.cash))
print('${} allowed for each ticker, or {}%.'.format(float(account.cash)/len(tickers),100/len(tickers)))
print("status: {}".format(account.status))
print("trading blocked: {}".format(account.trading_blocked))
print()
## check current positions ##
# Get a list of all of our positions.
portfolio = api.list_positions()
# Print the quantity of shares for each position.
for position in portfolio:
    print("I own {} shares of {}.".format(position.qty, position.symbol))

Account number: PA3A3BVCX4JY
Buying power: $17859.18
Cash available: $9646.59
$1205.82375 allowed for each ticker, or 12.5%.
status: ACTIVE
trading blocked: False

I own 10 shares of YINN.
I own -10 shares of TECL.
I own 10 shares of NAIL.


In [195]:
## loop that sets when logic is executed ##
minutes = time_to_market_close()
print(minutes)

1422.5002562166667


In [197]:

while minutes > 68:
    minutes = time_to_market_close()
    time.sleep(5)
    print(minutes)
    
print('it left the loop!!')
print()

## the start of the program will update us on a few things ##
account = api.get_account()
print("Account number: {}".format(account.account_number))
print("Buying power: ${}".format(account.buying_power))
print("Cash available: ${}".format(account.cash))
print('${} allowed for each ticker, or {}%.'.format(float(account.cash)/len(tickers),100/len(tickers)))
print("status: {}".format(account.status))
print("trading blocked: {}".format(account.trading_blocked))
print()
## check current positions ##
# Get a list of all of our positions.
portfolio = api.list_positions()
# Print the quantity of shares for each position.
for position in portfolio:
    print("I own {} shares of {}.".format(position.qty, position.symbol))

1422.2048600833334


KeyboardInterrupt: 

In [196]:
## print information after script has ran ##
## list orders and details ##
orders = api.list_orders()
print("{} orders in.".format(len(orders)))
for o in orders:
    print("Sell {} shares of {} on {}.".format(o.qty, o.symbol, o.time_in_force))


0 orders in.
